In [1]:
import os
import sys
import time
import matplotlib as mpl 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd

import tensorflow as tf
from tensorflow import keras

# print(tf.__version__)
# print(sys.version_info)
# for module in mpl,np,pd,sklearn,tf,keras:
#     print(module.__name__,module.__version__)

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all,y_train_all),(x_test,y_test) = fashion_mnist.load_data()
x_valid,x_train = x_train_all[:5000],x_train_all[5000:]
y_valid,y_train = y_train_all[:5000],y_train_all[5000:]

# print(x_valid.shape,y_valid.shape)
# print(x_train.shape,y_train.shape)
# print(x_test.shape,y_test.shape)
# print(np.max(x_train),np.min(x_train))

In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)
x_valid_scaled = scaler.transform(x_valid.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)
x_test_scaled = scaler.transform(x_test.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)

In [4]:
print(np.max(x_train_scaled),np.min(x_train_scaled))

2.0231433 -0.8105136


In [5]:
# 子类api
class WideDeepModel(keras.models.Model):
    def __init__(self):
        super(WideDeepModel,self).__init__()
        """定义模型的层次"""
        self.hidden1_layer = keras.layers.Dense(30,activation="relu")
        self.hidden2_layer = keras.layers.Dense(30,activation="relu")
        self.output_layer = keras.layers.Dense(1)
        
    def call(self,input):
        """完成模型的正向计算"""
        hidden1 = self.hidden1_layer(input)
        hidden2 = self.hidden2_layer(hidden1)
        concat = keras.layers.concatenate([input,hidden2])
        output = self.output_layer(concat)
        return output
model = WideDeepModel()
# model = keras.models.Sequential([
#     WideDeepModel()
# ])
model.build(input_shape=(None,8))
model.summary()

model.compile(loss="mean_squared_error",
              optimizer="sgd")


Model: "wide_deep_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  270       
_________________________________________________________________
dense_1 (Dense)              multiple                  930       
_________________________________________________________________
dense_2 (Dense)              multiple                  39        
Total params: 1,239
Trainable params: 1,239
Non-trainable params: 0
_________________________________________________________________


In [9]:

callbacks = [
    keras.callbacks.EarlyStopping(patience=5,min_delta=1e-2)
]

history=model.fit(x_train_scaled,y_train,
                  epochs = 100,
                  validation_data = (x_valid_scaled,y_valid),
                  callbacks = callbacks)

ValueError: ('Error when checking model target: expected no data, but got:', array([4, 0, 7, ..., 3, 0, 5], dtype=uint8))

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,3)
    plt.show()
    
plot_learning_curves(history)

In [ ]:
model.evaluate(x_test_scaled,y_test)